In [1]:
!pip install peft datasets numpy pydantic==1.10.9 py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
!pip install transformers accelerate

In [3]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.2 MB/s eta 0:00:00


In [4]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ec6a3e3c8b381f51ee35eca05dbe402bd91f926647eeea8f8ee2c44e0b709752
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
from datasets import load_dataset
dataset = load_dataset("EdinburghNLP/xsum")

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Train dataset size: 204045
Test dataset size: 11334


In [6]:
tokenized_datasets = dataset.filter(lambda example, index: index % 50 == 0, with_indices=True)

Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11332 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [7]:
dataset["train"]

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})

In [8]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [9]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import bitsandbytes
import torch

In [10]:
model_id = "Falconsai/text_summarization"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [11]:
def tokenize_function(example):
    start_prompt = 'Summarize the following text.\n\n'
    end_prompt = '\n\nSummary:'
    prompt = [start_prompt + document + end_prompt for document in example["document"]]

    model_inputs = tokenizer(prompt, padding="max_length", truncation=True)
    labels = tokenizer(text_target=example["summary"], padding="max_length", truncation=True)


    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [12]:
filter_data_train = dataset["train"].select(range(5000))
filter_data_test = dataset["test"].select(range(5000))

tokenized_datasets_train = filter_data_train.map(tokenize_function, batched=True)
tokenized_datasets_test = filter_data_test.map(tokenize_function, batched=True)
tokenized_datasets_train = tokenized_datasets_train.remove_columns(['document', 'summary','id'])
tokenized_datasets_test = tokenized_datasets_test.remove_columns(['document', 'summary','id'])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})

In [14]:
from peft import LoraConfig,get_peft_model,TaskType

In [15]:
lora_config = LoraConfig(
    r = 56,  #Rank of Low Rank Matrices
    lora_alpha = 32,#Similar to learning rate
    target_modules= ["q","k","v"],
    lora_dropout = 0.05,
    bias = "none",
    task_type = TaskType.SEQ_2_SEQ_LM

)

In [16]:
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 3,096,576 || all params: 63,603,200 || trainable%: 4.868585228416181


In [17]:
from transformers import GenerationConfig, TrainingArguments, Trainer

output_dir = "./content/"

peft_training_arguments = TrainingArguments(
    output_dir = output_dir,
    auto_find_batch_size = True,
    learning_rate = 1e-4,
    weight_decay = 0.01,
    num_train_epochs=3,
    logging_steps=50,
    evaluation_strategy="epoch"
)


peft_trainer =Trainer(
    model = model,
    args = peft_training_arguments,
    train_dataset = tokenized_datasets_train,
    eval_dataset = tokenized_datasets_test
)

In [18]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [19]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss
1,3.002400,2.759180
2,2.936900,2.730258
3,2.962800,2.724225


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


TrainOutput(global_step=1875, training_loss=3.0120358968098957, metrics={'train_runtime': 1693.8557, 'train_samples_per_second': 8.856, 'train_steps_per_second': 1.107, 'total_flos': 2172817244160000.0, 'train_loss': 3.0120358968098957, 'epoch': 3.0})

In [21]:
saved_dir = "summarization_model"
tokenizer.save_pretrained(saved_dir)
peft_trainer.model.save_pretrained(saved_dir)
peft_trainer.save_model(saved_dir)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
#Inference
from peft import PeftModel, PeftConfig
peft_model_id ="summarization_model"
config = PeftConfig.from_pretrained(peft_model_id)

#Laoding Base Model
# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path  )

# # # Load the Lora model
p_model = PeftModel.from_pretrained(model, peft_model_id ,is_trainable=False,device_map="auto")
p_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=512, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=56, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=56, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
            

Testing Data on the PEFT Model

In [33]:
index = 10
document = dataset['test'][index]['document']

prompt = f"""Summarise the following text
{document}"""

input_ids = tokenizer(prompt,return_tensors = "pt").input_ids.cuda()
peft_model_output = p_model.generate(input_ids = input_ids,generation_config=GenerationConfig(max_new_tokens=200, num_beams=2))


Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


In [34]:
peft_model_output

tensor([[    0,    37,  3819,    65,  2162,     3,     9,  3996,   927,    51,
          1340,    16,  3135,    12,  1340,     8,   381,    13, 12260,     7,
            44,     8,   636,    13,  5286,    31,     7,     5,     1]],
       device='cuda:0')

In [35]:
peft_model_text = tokenizer.decode(peft_model_output[0],skip_special_tokens =True)
print(f'PROMPT: \n {prompt}\n\n')
print("-" * 30)
print(f'PEFT MODEL SUMMARY:\n{peft_model_text}')

PROMPT: 
 Summarise the following text
The move is in response to an £8m cut in the subsidy received from the Department of Employment and Learning (DEL).
The cut in undergraduate places will come into effect from September 2015.
Job losses will be among both academic and non-academic staff and Queen's says no compulsory redundancies should be required.
There are currently around 17,000 full-time undergraduate and postgraduate students at the university, and around 3,800 staff.
Queen's has a current intake of around 4,500 undergraduates per year.
The university aims to reduce the number of student places by 1,010 over the next three years.
The BBC understands that there are no immediate plans to close departments or courses, but that the cuts in funding may put some departments and courses at risk.
The Education Minister Stephen Farry said he recognised that some students might now choose to study in other areas of the UK because of the cuts facing Northern Ireland's universities.
"Som

In [37]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.7 MB/s eta 0:00:00


In [38]:
import evaluate
import numpy as np

# Metric
metric = evaluate.load("rouge")

In [41]:
#Evaluate Function

def evaluate_peft_model(sample):
  outputs = p_model.generate(input_ids = sample["input_ids"].unsqueeze(0).cuda(),
                             generation_config=GenerationConfig(max_new_tokens=200, num_beams=2))
  predictions =tokenizer.decode(outputs[0], skip_special_tokens=True)
  labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
  labels = tokenizer.decode(labels, skip_special_tokens=True)

  return predictions, labels


In [44]:
filter_data_val = dataset["validation"].select(range(100))

tokenized_datasets_val = filter_data_val.map(tokenize_function, batched=True)
eval_dataset = tokenized_datasets_val.with_format("torch")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [46]:
predictions,references = [],[]
for sample in eval_dataset:
  p,l = evaluate_peft_model(sample)
  predictions.append(p)
  references.append(l)

In [47]:
# compute metric
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")

Rogue1: 7.021154%
rouge2: 1.524679%
rougeL: 5.677334%
rougeLsum: 5.615002%
